# <span style="font-width:bold; font-size: 3rem; color:#2656a3;">**Msc. BDS - M7 Second Semester Project** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Feature Pipeline</span>

## <span style='color:#2656a3'> 🗒️ The notebook is divided into the following sections:
1. Parsing new data.
2. Inserting the new data into the Feature Store.

## <span style='color:#2656a3'> ⚙️ Import of libraries and packages

We start by accessing the folder we have created that holds the functions (incl. live API calls and data preprocessing) we need for electricity prices and weather measures. Then, we proceed to import some of the necessary libraries needed for this notebook and warnings to avoid unnecessary distractions and keep output clean.

In [1]:
# First we go one back in our directory to access the folder with our functions
%cd ..

# Now we import the functions from the features folder
# This is the functions we have created to generate features for electricity prices and weather measures
from features import electricity_prices, weather_measures

# We go back into the notebooks folder
%cd pipeline

/Users/tobiasmjensen/Documents/aau_bds/m7_second-semester-project/bds_m7_second-semester-project
/Users/tobiasmjensen/Documents/aau_bds/m7_second-semester-project/bds_m7_second-semester-project/pipeline


In [2]:
# Importing pandas for data handling
import pandas as pd
from datetime import datetime, timedelta

# Ignore warnings
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning)

## <span style='color:#2656a3'> 🪄 Parsing New Data
To fetch non-historical electricity prices we are setting `historical` to `False`. 

In order to provide real time weather measures, a weather forecast measure for the next 5 days is being fetched.

There are of course no changes to the calendar data, and therefore no new data is retrieved from it.

### <span style="color:#2656a3;">💸 Electricity Prices per day from Energinet

In [3]:
# Fetching non-historical electricity prices for area DK1
electricity_price_df = electricity_prices.electricity_prices(
    historical=False,
    area=["DK1"]
)

In [4]:
# Display the electricity dataframe
electricity_price_df

,timestamp,datetime,date,hour,dk1_spotpricedkk_kwh
0,1716595200000,2024-05-25 00:00:00,2024-05-25,0,0.70789
1,1716598800000,2024-05-25 01:00:00,2024-05-25,1,0.68513
2,1716602400000,2024-05-25 02:00:00,2024-05-25,2,0.67469
3,1716606000000,2024-05-25 03:00:00,2024-05-25,3,0.66521
4,1716609600000,2024-05-25 04:00:00,2024-05-25,4,0.66424
5,1716613200000,2024-05-25 05:00:00,2024-05-25,5,0.67342
6,1716616800000,2024-05-25 06:00:00,2024-05-25,6,0.67655
7,1716620400000,2024-05-25 07:00:00,2024-05-25,7,0.64335
8,1716624000000,2024-05-25 08:00:00,2024-05-25,8,0.58628
9,1716627600000,2024-05-25 09:00:00,2024-05-25,9,0.55770


In [5]:
# Display the information of the electricity dataframe
electricity_price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   timestamp             24 non-null     int64         
 1   datetime              24 non-null     datetime64[ns]
 2   date                  24 non-null     datetime64[ns]
 3   hour                  24 non-null     int64         
 4   dk1_spotpricedkk_kwh  24 non-null     float64       
dtypes: datetime64[ns](2), float64(1), int64(2)
memory usage: 1.1 KB


### <span style="color:#2656a3;">🪟 Rolling window for Electricity Prices

In [6]:
# Fetching non-historical electricity prices and rolling window for area DK1
electricity_price_window_df = electricity_prices.electricity_prices_window(
    historical=False,
    area=["DK1"],
    end=datetime.now().date() + timedelta(days=(7*1)) 
)

In [7]:
# Display the electricity_window dataframe
electricity_price_window_df

,timestamp,datetime,prev_1w_mean,prev_2w_mean,prev_4w_mean,prev_6w_mean,prev_8w_mean,prev_12w_mean
38544,1716595200000,2024-05-25 00:00:00,0.483995,0.340337,0.396817,0.440260,0.419892,0.432834
38545,1716598800000,2024-05-25 01:00:00,0.485406,0.339550,0.396420,0.440505,0.419882,0.432842
38546,1716602400000,2024-05-25 02:00:00,0.486800,0.338797,0.396047,0.440800,0.419868,0.432849
38547,1716606000000,2024-05-25 03:00:00,0.488181,0.337964,0.395646,0.441108,0.419853,0.432859
38548,1716609600000,2024-05-25 04:00:00,0.489582,0.337203,0.395259,0.441418,0.419836,0.432871
...,...,...,...,...,...,...,...,...
38731,1717268400000,2024-06-01 19:00:00,NaN,0.515100,0.411789,0.442645,0.422362,0.426689
38732,1717272000000,2024-06-01 20:00:00,NaN,0.512304,0.410551,0.442302,0.422425,0.426689
38733,1717275600000,2024-06-01 21:00:00,NaN,0.509967,0.409645,0.442019,0.422509,0.426690
38734,1717279200000,2024-06-01 22:00:00,NaN,0.508518,0.409097,0.441773,0.422619,0.426697


In [8]:
# Display the information of the electricity_window dataframe
electricity_price_window_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 38544 to 38735
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   timestamp      192 non-null    int64         
 1   datetime       192 non-null    datetime64[ns]
 2   prev_1w_mean   167 non-null    float64       
 3   prev_2w_mean   192 non-null    float64       
 4   prev_4w_mean   192 non-null    float64       
 5   prev_6w_mean   192 non-null    float64       
 6   prev_8w_mean   192 non-null    float64       
 7   prev_12w_mean  192 non-null    float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 13.5 KB


### <span style="color:#2656a3;"> 🌈 Forecast Weather Measures from Open Meteo

In [9]:
# Fetching weather forecast measures for the next 5 days
forecast_weather_df = weather_measures.forecast_weather_measures(
    forecast_length=5
)

In [10]:
# Display the weather forecast dataframe
forecast_weather_df

,timestamp,datetime,date,hour,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
0,1716595200000,2024-05-25 00:00:00,2024-05-25,0,13.1,88.0,0.0,0.0,0.0,1.0,24.0,2.9,5.0
1,1716598800000,2024-05-25 01:00:00,2024-05-25,1,12.4,85.0,0.0,0.0,0.0,1.0,42.0,5.0,8.3
2,1716602400000,2024-05-25 02:00:00,2024-05-25,2,11.9,84.0,0.0,0.0,0.0,1.0,22.0,4.3,7.6
3,1716606000000,2024-05-25 03:00:00,2024-05-25,3,11.7,85.0,0.0,0.0,0.0,2.0,67.0,4.0,7.9
4,1716609600000,2024-05-25 04:00:00,2024-05-25,4,11.5,85.0,0.0,0.0,0.0,2.0,68.0,3.2,6.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,1717009200000,2024-05-29 19:00:00,2024-05-29,19,14.2,92.0,0.0,0.0,0.0,3.0,96.0,10.8,26.6
116,1717012800000,2024-05-29 20:00:00,2024-05-29,20,13.8,93.0,0.0,0.0,0.0,3.0,91.0,9.7,22.3
117,1717016400000,2024-05-29 21:00:00,2024-05-29,21,13.5,94.0,0.0,0.0,0.0,3.0,87.0,8.6,18.4
118,1717020000000,2024-05-29 22:00:00,2024-05-29,22,13.2,95.0,0.0,0.0,0.0,2.0,84.0,7.3,16.2


In [11]:
# Display the information of the weather forecast dataframe
forecast_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   timestamp             120 non-null    int64         
 1   datetime              120 non-null    datetime64[ns]
 2   date                  120 non-null    datetime64[ns]
 3   hour                  120 non-null    int64         
 4   temperature_2m        120 non-null    float64       
 5   relative_humidity_2m  120 non-null    float64       
 6   precipitation         120 non-null    float64       
 7   rain                  120 non-null    float64       
 8   snowfall              120 non-null    float64       
 9   weather_code          120 non-null    float64       
 10  cloud_cover           120 non-null    float64       
 11  wind_speed_10m        120 non-null    float64       
 12  wind_gusts_10m        120 non-null    float64       
dtypes: datetime64[ns](2)

## <span style="color:#2656a3;"> 📡 Connecting to Hopsworks Feature Store

We connect to Hopsworks Feature Store so we can access the Feature Groups and upload the new data into the Feature Groups.

In [12]:
# Importing the hopsworks module for interacting with the Hopsworks platform
import hopsworks

# Logging into the Hopsworks project
project = hopsworks.login()

# Getting the feature store from the project
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/554133
Connected. Call `.close()` to terminate connection gracefully.


In [13]:
# Retrieve the feature groups
electricity_price_fg = fs.get_feature_group(
    name="electricity_spot_prices",
    version=1,
)

# Retrieve the feature groups
electricity_price_window_fg = fs.get_feature_group(
    name="electricity_spot_price_window",
    version=1,
)

weather_fg = fs.get_feature_group(
    name="weather_measurements",
    version=1,
)

### <span style="color:#2656a3;"> ⬆️ Uploading new data to the Feature Store
Here we upload the new data to the retrieved Feature groups by using the `insert` function.

In [14]:
# Inserting the electricity_price_df into the feature group named electricity_fg
electricity_price_fg.insert(electricity_price_df, 
                      write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/24 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: electricity_spot_prices_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/554133/jobs/named/electricity_spot_prices_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x3010a1650>, None)

In [15]:
# Inserting the electricity_price_window_df into the feature group named electricity_price_window_fg
electricity_price_window_fg.insert(electricity_price_window_df, 
                      write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/192 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: electricity_spot_price_window_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/554133/jobs/named/electricity_spot_price_window_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x3010cff10>, None)

In [16]:
# Inserting the weather_df into the feature group named weather_fg
weather_fg.insert(forecast_weather_df, 
                  write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/120 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_measurements_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/554133/jobs/named/weather_measurements_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x3010e20d0>, None)

---
## <span style="color:#2656a3;">⏭️ **Next:** Part 03: Training Pipeline </span>

Next we will create a feature view and training dataset. Further we will train a model and save it in model registry.